In [ ]:
# -*- coding: utf-8 -*-
# """
# Created on Fri Jul 29 14:25:21 2022
# ​
# @author: Magnusson
# """
# ​
import scipy
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from spectral_connectivity import Multitaper
from spectral_connectivity import Connectivity
from spectral_connectivity import multitaper_connectivity
from pathlib import Path


pathsHPC = [path for path in Path('C:/Users/LENOVO/Desktop/sem7/internship/HomeCageHC').resolve().glob("**/*HPC*.mat")] # these paths are according to my local machine
pathsPFC = [path for path in Path('C:/Users/LENOVO/Desktop/sem7/internship/HomeCageHC').resolve().glob("**/*PFC*.mat")] # these paths are according to my local machine

frequency_of_interest = list(range(10,30))
sampling_frequency = 1000
n_trials = 40
n_signals = 2
no_of_files = len(pathsHPC)
# if len(pathsPFC) == len(pathsHPC):
#     no_of_files = len(pathsHPC)








In [ ]:
def plot_directional(time_series, time_halfbandwidth_product,k,p):
    m = Multitaper(
        time_series,
        sampling_frequency=sampling_frequency,
        time_halfbandwidth_product=time_halfbandwidth_product,
        start_time=0
    )
    c = Connectivity(
        fourier_coefficients=m.fft(), frequencies=m.frequencies, time=m.time
    )

    measures = dict(
        pairwise_spectral_granger=c.pairwise_spectral_granger_prediction(),
        directed_transfer_function=c.directed_transfer_function(),
        partial_directed_coherence=c.partial_directed_coherence(),
        generalized_partial_directed_coherence=c.generalized_partial_directed_coherence(),
        direct_directed_transfer_function=c.direct_directed_transfer_function(),
    )

    n_signals = 2 #time_series.shape[-1]
    signal_ind2, signal_ind1 = np.meshgrid(np.arange(n_signals), np.arange(n_signals))

    fig, axes = plt.subplots(
        n_signals, n_signals, figsize=(n_signals * 2, n_signals * 2), sharex=True
    )
    measure_name, measure = list(measures.items())[k]

    for ind1, ind2, ax in zip(signal_ind1.ravel(), signal_ind2.ravel(), axes.ravel()):
        # for measure_name, measure in measures.items():
        ax.plot(
            c.frequencies[0:27337],
            measure[0, 0:27337, ind1, ind2],
            label=measure_name,
            linewidth=5,
            alpha=0.8,
            )
        ax.set_title("x{} → x{}".format(ind2 + 1, ind1 + 1), fontsize=15)
        ax.set_ylim((0, np.max([np.nanmax(np.stack(list(measures.values()))), 1.05])))
        ax.set_xlim([0,90])

    axes[0, -1].legend()
    plt.tight_layout()
    plt.savefig(f'C:/Users/LENOVO/Desktop/sem7/internship/trial{p}_{measure_name}.png', dpi=300, bbox_inches='tight')
    

In [ ]:
for i in range(1): 
    dataHPC = scipy.io.loadmat(pathsHPC[i])
    dataPFC = scipy.io.loadmat(pathsPFC[i])
   
    
    current_dataHPC = dataHPC['HPC']
    current_dataPFC = dataPFC['PFC']

    n_time_samples = len(current_dataHPC)
    time = np.arange(0, n_time_samples)/sampling_frequency 
    signal = np.zeros((n_time_samples, n_signals))
    signal[:, 0] = np.reshape(current_dataHPC, (n_time_samples,))
    signal[:, 1] = np.reshape(current_dataPFC, (n_time_samples,))
#need to specify all this
    m = Multitaper(signal,
                   sampling_frequency=sampling_frequency,
                   time_halfbandwidth_product=5,
                   start_time=time[0])
    c = Connectivity(fourier_coefficients=m.fft(),
                     frequencies=m.frequencies)
    for k in range(5):
        plot_directional(signal, time_halfbandwidth_product = 5, k = k ,p=i)

    fig, axes = plt.subplots(1, 1, figsize=(3, 3), sharex=True, sharey=True)
    axes.plot(c.frequencies, c.power().squeeze())
    axes.set_xlim([0,90])
    plt.savefig(f'C:/Users/LENOVO/Desktop/sem7/internship/trial{i}_power.png', dpi=300, bbox_inches='tight')


